# Data Project 1 - Ryan Wood

In [ ]:
# import statements, don't know if I'll need all of them

import os
import csv
import json
import numpy
import datetime
import pandas as pd
import urllib.request
import pprint

import pymongo
from sqlalchemy import create_engine
import matplotlib.pyplot as plt

In [ ]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id = "root2"
pwd = "123456789"

src_dbname = "dp1updated"
dst_dbname = "dp1mongoupdates"

### Define Functions for Getting Data From and Setting Data Into Databases

In [ ]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [ ]:
try:
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)

    sqlEngine.execute(f"DROP DATABASE IF EXISTS `{src_dbname}`;")
    sqlEngine.execute(f"CREATE DATABASE `{src_dbname}`;")
    sqlEngine.execute(f"USE {src_dbname};")
except:
    print("There was an error creating a sql engine from the information provided in the program")

### CSV to SQL conversion

In [ ]:
try:
    data_dir = os.path.join(os.getcwd(), 'data')
    data_file = os.path.join(data_dir, 'superbowl.csv')
    # df = pd.read_csv(data_file, header=0, index_col=0)
    df = pd.read_csv(data_file)
    total_rows = len(df.axes[0]) #===> Axes of 0 is for a row
    total_cols = len(df.axes[1]) #===> Axes of 1 is for a column
    print("Number of Rows: " + str(total_rows))
    print("Number of Columns: " + str(total_cols))
    df.head()
except:
    print("The file could not be read in correctly.")
    print("Make sure the data file is in the correct directory so it can be inserted properly.")

In [ ]:
# Add Super Bowl 55
new_row = pd.DataFrame({'Date':'Feb 7 2021', 'SB':'LV (55)', 'Winner':'Tampa Bay Buccaneers', 'Winner Pts':'31',
                        'Loser':'Kansas City Chiefs', 'Loser Pts':'9', 'MVP':'Tom Brady',
                        'Stadium':'Raymond James Stadium', 'City':'Tampa', 'State':'Florida'}, index = [0])
df = pd.concat([new_row, df]).reset_index(drop = True)
df.head(5)

In [ ]:
# Add Super Bowl 56
new_row2 = pd.DataFrame({'Date':'Feb 13 2022', 'SB':'LVI (56)', 'Winner':'Los Angeles Rams', 'Winner Pts':'23', 
                          'Loser':'Cincinnati Bengals', 'Loser Pts':'20','MVP':'Cooper Kupp', 'Stadium':'SoFi Stadium', 
                            'City':'Inglewood', 'State':'California'}, index = [0])
df = pd.concat([new_row2, df]).reset_index(drop = True)

df.head(5)

In [ ]:
# drop useless column
df.drop('State', axis=1, inplace=True)
df.head(5)

In [ ]:
start_val = 1
df.insert(loc=0, column = 'surr_key', value = range(start_val, len(df) + start_val))
df.head(5)

In [ ]:
set_dataframe(user_id, pwd, host_name, "dp1updated", df, "superbowls", "surr_key", "insert")

### Convert from SQL to MongoDB Database

In [ ]:
# host_name = "localhost"
ports = {"mongo" : 27017, "mysql" : 3306}

# user_id = "root2"
# pwd = "123456789"
 
src_dbname = "dp1updated"
dst_dbname = "dp1mongoupdates"

In [ ]:
def get_sql_dataframe(user_id, pwd, host_name, db_name, sql_query):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    
    '''Invoke the pd.read_sql() function to query the database, and fill a Pandas DataFrame.'''
    conn = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, conn);
    conn.close()
    
    return dframe


def get_mongo_dataframe(user_id, pwd, host_name, port, db_name, collection, query):
    '''Create a connection to MongoDB, with or without authentication credentials'''
    if user_id and pwd:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db_name)
        client = pymongo.MongoClient(mongo_uri)
    else:
        conn_str = f"mongodb://{host_name}:{port}/"
        client = pymongo.MongoClient(conn_str)
    
    '''Query MongoDB, and fill a python list with documents to create a DataFrame'''
    db = client[db_name]
    dframe = pd.DataFrame(list(db[collection].find(query)))
    dframe.drop(['_id'], axis=1, inplace=True)
    client.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    '''Create a connection to the MySQL database'''
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    '''Invoke the Pandas DataFrame .to_sql( ) function to either create, or append to, a table'''
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

#### RUN THIS ONLY ONCE (OR ELSE MONGO WILL HAVE DUPLICATE ENTRIES)

In [ ]:
# only run this when you are going back through the project again after running the cell below once
port = ports["mongo"]
conn_str = f"mongodb://{host_name}:{port}/"
client = pymongo.MongoClient(conn_str)
db = client[src_dbname]

In [ ]:
# NOTE FOR MYSELF: DO NOT RUN ANY MORE, ALREADY IN MONGO

port = ports["mongo"]
conn_str = f"mongodb://{host_name}:{port}/"
client = pymongo.MongoClient(conn_str)
db = client[src_dbname]

data_dir = os.path.join(os.getcwd(), 'data')

json_files = {"SBstats" : 'superbowl_json.json'
             }

for file in json_files:
    json_file = os.path.join(data_dir, json_files[file])
    with open(json_file, 'r') as openfile:
        json_object = json.load(openfile)
        file = db[file]
        result = file.insert_many(json_object)
        #print(f"{file} was successfully loaded.")

        
# client.close()     

#### Transformations within MongoDB

In [ ]:
query = {}
port = ports["mongo"]
collection = "SBstats"

df_mongo_orig = get_mongo_dataframe(None, None, host_name, port, src_dbname, collection, query)
df_mongo_orig.head(5)

In [ ]:
sql_cowboys = "SELECT * FROM dp1updated.superbowls WHERE `Winner` = 'Dallas Cowboys';"
df_cowboys_sb = get_sql_dataframe(user_id, pwd, host_name, src_dbname, sql_cowboys)
df_cowboys_sb.head(5)

In [ ]:
db.list_collection_names()

In [ ]:
collection = "SBstats"

stats = db[collection]

In [ ]:
# The SELECT list -----------------------------------------------
projection = {"_id": 0, "Date": 1, "SB": 1, "Winner": 1, "Winner Pts": 1, "Loser": 1, "Loser Pts": 1, "MVP": 1,
             "Stadium": 1, "City": 1, "State": 1}

# The WHERE clause ----------------------------------------------
conditions = {"Winner":{"$eq": "Dallas Cowboys"}}

# The ORDER BY clause -------------------------------------------
orderby = [("Date", -1)]

for title in stats.find(conditions, projection).sort(orderby):
    print(title)

In [ ]:
df_cowboys_sb_mongo = pd.DataFrame( list( stats.find(conditions, projection).sort(orderby) ) )
df_cowboys_sb_mongo.head(5)

#### Do more things in MongoDB before pushing back to new SQL Schema from Modified SQL Schema

In [ ]:
# NOTE FOR MYSELF: DO NOT RUN ANY MORE, ALREADY IN MONGO

port = ports["mongo"]
conn_str = f"mongodb://{host_name}:{port}/"
client = pymongo.MongoClient(conn_str)
db2 = client[dst_dbname]

data_dir = os.path.join(os.getcwd(), 'data')

json_files = {"updated_sb_stats" : 'updated_superbowls_as_json.json'
             }

for file in json_files:
    json_file = os.path.join(data_dir, json_files[file])
    with open(json_file, 'r') as openfile:
        json_object = json.load(openfile)
        file = db2[file]
        result = file.insert_many(json_object)
        print(f"{file} was successfully loaded.")

        
client.close()

In [ ]:
query = {}
port = ports["mongo"]
collection = "updated_sb_stats"

conn_str = f"mongodb://{host_name}:{port}/"
client = pymongo.MongoClient(conn_str)
db2 = client[dst_dbname]

try:
    df_mongo_updated = get_mongo_dataframe(None, None, host_name, port, dst_dbname, collection, query)
    df_mongo_updated.head(5)
except:
    print("There was an error getting the MongoDB dataframe.")
    print("Make sure the dataframe exists and other arguments are correct.")    

In [ ]:
sql_cowboys2 = "SELECT * FROM dp1updated.superbowls WHERE `Winner` = 'Dallas Cowboys';"
df_cowboys_sb2 = get_sql_dataframe(user_id, pwd, host_name, src_dbname, sql_cowboys2)
df_cowboys_sb2.head(5)

In [ ]:
db2.list_collection_names()

In [ ]:
collection2 = "updated_sb_stats"

stats2 = db2[collection2]

In [ ]:
# The SELECT list -----------------------------------------------
projection2 = {"_id": 0, "surr_key": 1, "Date": 1, "SB": 1, "Winner": 1, "Winner Pts": 1, "Loser": 1,
               "Loser Pts": 1, "MVP": 1, "Stadium": 1, "City": 1}

# The WHERE clause ----------------------------------------------
conditions = {"Winner":{"$eq": "Dallas Cowboys"}}

# The ORDER BY clause -------------------------------------------
orderby = [("Date", 1)]

for title in stats2.find(conditions, projection2).sort(orderby):
    print(title)

In [ ]:
df_cowboys_sb_mongo_updated = pd.DataFrame( list( stats2.find(conditions, projection2).sort(orderby) ) )
df_cowboys_sb_mongo_updated.head(5)

In [ ]:
# insert into sql schema again

dataframe = df_cowboys_sb_mongo_updated
table_name = 'updated_sb_stats'
primary_key = 'surr_key'
db_operation = "insert"

try:
    set_dataframe(user_id, pwd, host_name, src_dbname, dataframe, table_name, primary_key, db_operation)
except:
    print("There was an error setting the new dataframe.")
    print("Make sure the destination database, primary key, and other arguments are correct.")    

In [ ]:
client.close()